In [9]:
import pandas as pd
from pathlib import Path

In [10]:
# df = pd.read_excel('letterboxd.xlsx')
# df['URL'] = df['URL'].str.replace('https://boxd.it/', '')
# df['Year'] = df['Year'].astype(int)
# print(df.head())

In [11]:
def clean_title(title):
    if title[:4] == "The ":
        title = title[4:] + ", The"
    elif title[:2] == "A ":
        title = title[2:] + ", A"
    elif title[:3] == "An ":
        title = title[3:] + ", An"
    
    return title
    


In [12]:
lists_folder = Path("lists")
csv_files = lists_folder.glob("*.csv")

dfs = []

for file in csv_files:
    df = pd.read_csv(file, skiprows=4)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['URL'] = combined_df['URL'].str.replace('https://boxd.it/', '')
combined_df['Year'] = combined_df['Year'].astype(int)
combined_df['Name'] = combined_df['Name'].apply(clean_title)

C:\Users\Tom\AppData\Local\Temp\ipykernel_21416\174825041.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_df['URL'] = combined_df['URL'].str.replace('https://boxd.it/', '')


In [13]:
combined_df.head()

,Position,Name,Year,URL,Description
0,1,All of Us Strangers,2023,Bz3C,NaN
1,2,"Zone of Interest, The",2023,gJsA,NaN
2,3,"Boy and the Heron, The",2023,ipeM,NaN
3,4,Poor Things,2023,tNWU,NaN
4,5,Oppenheimer,2023,wUow,NaN


In [14]:
film_counts = (
    combined_df.groupby("URL")
      .agg(
          appearances=("URL", "count"),
          name=("Name", "first"),
          year=("Year", "first")
      )
      .reset_index()
)

In [15]:
film_counts.head()

,URL,appearances,name,year
0,10hC,1,Make Way for Tomorrow,1937
1,10iU,3,La Notte,1961
2,10iq,1,Pyaasa,1957
3,11PU,1,Black Orpheus,1959
4,11aM,3,"Heartbreak Kid, The",1972


In [16]:
film_count_counts = (
    film_counts["appearances"]
    .value_counts()
    .sort_index(ascending=False)
    .rename("number of films")
    .to_frame()
)

film_count_counts["cumulative total"] = film_count_counts["number of films"].cumsum()

print(film_count_counts)

    number of films  cumulative total
10               18                18
9                27                45
8                30                75
7                35               110
6                41               151
5                71               222
4               111               333
3               187               520
2               422               942
1              1264              2206


In [24]:
filtered_films = film_counts[film_counts["appearances"] == 7].sort_values(by=["name"], ascending=True).reset_index()

In [22]:
filtered_films.head()

,index,URL,appearances,name,year
0,1597,2auI,9,12 Angry Men,1957
1,1673,2bf0,10,2001: A Space Odyssey,1968
2,1589,2aqm,8,8½,1963
3,1605,2awY,10,Alien,1979
4,1279,29PA,7,Aliens,1986


In [25]:
for i, row in filtered_films.iterrows():
    print(f"{i+1}. {row['name']} ({row['year']}) - {row['appearances']}")

1. Aliens (1986) - 7
2. All That Jazz (1979) - 7
3. Brazil (1985) - 7
4. Brief Encounter (1945) - 7
5. Children of Men (2006) - 7
6. City Lights (1931) - 7
7. Cléo from 5 to 7 (1962) - 7
8. Dark Knight, The (2008) - 7
9. Die Hard (1988) - 7
10. Empire Strikes Back, The (1980) - 7
11. Eternal Sunshine of the Spotless Mind (2004) - 7
12. Everything Everywhere All at Once (2022) - 7
13. Fight Club (1999) - 7
14. Get Out (2017) - 7
15. Ghostbusters (1984) - 7
16. Grand Budapest Hotel, The (2014) - 7
17. Handmaiden, The (2016) - 7
18. High and Low (1963) - 7
19. His Girl Friday (1940) - 7
20. Inglourious Basterds (2009) - 7
21. Matter of Life and Death, A (1946) - 7
22. Modern Times (1936) - 7
23. Network (1976) - 7
24. No Country for Old Men (2007) - 7
25. Oldboy (2003) - 7
26. Oppenheimer (2023) - 7
27. Rashomon (1950) - 7
28. Red Shoes, The (1948) - 7
29. Seventh Seal, The (1957) - 7
30. Social Network, The (2010) - 7
31. Some Like It Hot (1959) - 7
32. Tokyo Story (1953) - 7
33. Umbrell

In [9]:
selection = film_counts[film_counts["appearances"] == 6].sort_values(by="name", ascending=True)
for _, row in selection.iterrows():
    print(f"{row['name']} ({row['year']})")

A Clockwork Orange (1971)
Aliens (1986)
All That Jazz (1979)
Brazil (1985)
Children of Men (2006)
Children of Paradise (1945)
Chungking Express (1994)
Everything Everywhere All at Once (2022)
Fanny and Alexander (1982)
Get Out (2017)
High and Low (1963)
Inception (2010)
Interstellar (2014)
Last Year at Marienbad (1961)
Metropolis (1927)
Moonlight (2016)
No Country for Old Men (2007)
Oldboy (2003)
Oppenheimer (2023)
Saving Private Ryan (1998)
Spider-Man: Into the Spider-Verse (2018)
Suspiria (1977)
Taste of Cherry (1997)
The 400 Blows (1959)
The Elephant Man (1980)
The General (1926)
The Handmaiden (2016)
The Life and Death of Colonel Blimp (1943)
The Lives of Others (2006)
The Piano (1993)
The Red Shoes (1948)
The Seventh Seal (1957)
The Social Network (2010)
The Texas Chain Saw Massacre (1974)
The Treasure of the Sierra Madre (1948)
The Umbrellas of Cherbourg (1964)
The Young Girls of Rochefort (1967)
To Be or Not to Be (1942)
When Harry Met Sally... (1989)
Whiplash (2014)
